In [15]:
%%file /tmp/ex.trs
(VAR x y)
(RULES
    plus(0,y) -> y
    plus(s(x),y) -> s(plus(x,y))
)

Writing /tmp/ex.trs


In [23]:
%%file /tmp/append.trs
(VAR x y z)
(RULES
    append(nil,y) -> y
    append(cons(x,z),y) -> cons(x, append(z,y))
)

Writing /tmp/append.trs


In [25]:
! java -ea -jar ~/Downloads/term-rewrite/aprove.jar -m wst /tmp/append.trs -p cpf

YES
<?xml version="1.0" encoding="UTF-8" standalone="no"?><?xml-stylesheet type="text/xsl" href="cpfHTML.xsl"?><certificationProblem xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="cpf.xsd"><input><trsInput><trs><rules><rule><lhs><funapp><name>append</name><arg><funapp><name>nil</name></funapp></arg><arg><var>y</var></arg></funapp></lhs><rhs><var>y</var></rhs></rule><rule><lhs><funapp><name>append</name><arg><funapp><name>cons</name><arg><var>x</var></arg><arg><var>z</var></arg></funapp></arg><arg><var>y</var></arg></funapp></lhs><rhs><funapp><name>cons</name><arg><var>x</var></arg><arg><funapp><name>append</name><arg><var>z</var></arg><arg><var>y</var></arg></funapp></arg></funapp></rhs></rule></rules></trs></trsInput></input><cpfVersion>2.1</cpfVersion><proof><trsTerminationProof><ruleRemoval><orderingConstraintProof><redPair><knuthBendixOrder><w0>1</w0><precedenceWeight><precedenceWeightEntry><name>nil</name><arity>0</arity><precedence>0</precede

In [ ]:
import subprocess
res = subprocess.run(["java", "-ea", "-jar", "/home/philip/Downloads/aprove.jar", "/tmp/ex.trs", "-p", "cpf"], capture_output=True, check=True)
print(res)
print(dir(res.stdout))
import xml.etree.ElementTree as ET
tree = ET.fromstring(res.stdout.decode())
print(tree)
print(tree.tag)

In [ ]:
tree.tag
tree.attrib
print([elem.tag for elem in tree])
input = tree.find("input")
print(dir(input))
input.text
ET.tostring(input)
ET.dump(input)

In [ ]:
def etree_to_dict(t):
    d = {t.tag : map(etree_to_dict, t.getchildren())}
    d.update(('@' + k, v) for k, v in t.attrib.iteritems())
    d['text'] = t.text
    return d

ebpf termination via C prover
https://qmonnet.github.io/whirl-offload/2020/04/12/llvm-ebpf-asm/
https://mdaverde.com/posts/ebpf-dump-insns/

In [20]:
%%file /tmp/fact.c

int fact(int x){
    int y = 1;
    while(x > 1){
        x = x - 1;
        y = y * x;
    }
    return y;
}

Overwriting /tmp/fact.c


In [22]:
! java -ea -cp ~/Downloads/term-rewrite/aprove.jar aprove.CommandLineInterface.CFrontendMain /tmp/fact.c

line 7:31 mismatched input 'noundef' expecting CLOSEP
line 7:41 mismatched input ')' expecting ASSIGN
line 8:7 mismatched input 'alloca' expecting TYPE
line 9:7 mismatched input 'alloca' expecting TYPE
line 10:14 mismatched input ',' expecting ASSIGN
line 10:22 mismatched input ',' expecting ASSIGN
line 11:21 mismatched input ',' expecting ASSIGN
line 14:0 mismatched input '4' expecting ASSIGN
line 15:7 mismatched input 'load' expecting TYPE
line 15:23 mismatched input ',' expecting ASSIGN
line 16:7 mismatched input 'icmp' expecting TYPE
line 16:22 mismatched input ',' expecting ASSIGN
line 17:10 mismatched input ',' expecting ASSIGN
line 17:20 mismatched input ',' expecting ASSIGN
line 19:0 mismatched input '7' expecting ASSIGN
line 20:7 mismatched input 'load' expecting TYPE
line 20:23 mismatched input ',' expecting ASSIGN
line 21:7 mismatched input 'sub' expecting TYPE
line 21:21 mismatched input ',' expecting ASSIGN
line 22:14 mismatched input ',' expecting ASSIGN
line 22:22 mismat

In [5]:
!clang -target bpf -O2 -c /tmp/fact.c -o /tmp/fact.o

In [10]:
!llvm-objdump -d /tmp/fact.o


/tmp/fact.o:	file format elf64-bpf

Disassembly of section .text:

0000000000000000 <fact>:
       0:	b7 00 00 00 01 00 00 00	r0 = 0x1
       1:	bf 12 00 00 00 00 00 00	r2 = r1
       2:	67 02 00 00 20 00 00 00	r2 <<= 0x20
       3:	c7 02 00 00 20 00 00 00	r2 s>>= 0x20
       4:	b7 03 00 00 02 00 00 00	r3 = 0x2
       5:	6d 23 09 00 00 00 00 00	if r3 s> r2 goto +0x9 <LBB0_3>
       6:	07 01 00 00 01 00 00 00	r1 += 0x1

0000000000000038 <LBB0_2>:
       7:	bf 12 00 00 00 00 00 00	r2 = r1
       8:	07 02 00 00 fe ff ff ff	r2 += -0x2
       9:	2f 20 00 00 00 00 00 00	r0 *= r2
      10:	07 01 00 00 ff ff ff ff	r1 += -0x1
      11:	bf 12 00 00 00 00 00 00	r2 = r1
      12:	67 02 00 00 20 00 00 00	r2 <<= 0x20
      13:	77 02 00 00 20 00 00 00	r2 >>= 0x20
      14:	25 02 f8 ff 02 00 00 00	if r2 > 0x2 goto -0x8 <LBB0_2>

0000000000000078 <LBB0_3>:
      15:	95 00 00 00 00 00 00 00	exit


In [11]:
! # angr decompile /tmp/fact.o  deosn't work

Traceback (most recent call last):
  File "/home/philip/.local/bin/angr", line 8, in <module>
    sys.exit(main())
  File "/home/philip/.local/lib/python3.10/site-packages/angr/__main__.py", line 50, in main
    decompilation = decompile_functions(
  File "/home/philip/.local/lib/python3.10/site-packages/angr/analyses/decompiler/utils.py", line 651, in decompile_functions
    proj = angr.Project(path, auto_load_libs=False)
  File "/home/philip/.local/lib/python3.10/site-packages/angr/project.py", line 147, in __init__
    self.loader = cle.Loader(self.filename, concrete_target=concrete_target, **load_options)
  File "/home/philip/.local/lib/python3.10/site-packages/cle/loader.py", line 188, in __init__
    self.initial_load_objects = self._internal_load(
  File "/home/philip/.local/lib/python3.10/site-packages/cle/loader.py", line 782, in _internal_load
    obj = self._load_object_isolated(main_spec)
  File "/home/philip/.local/lib/python3.10/site-packages/cle/loader.py", line 985, in 